In [ ]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SparkSession
import org.bdgenomics.adam.rdd.ADAMContext._
import comp.bio.aging.playground.extensions._
import org.apache.spark.sql._
import scala.util.Try
import org.apache.spark.storage.StorageLevel

val spark = SparkSession
    .builder()
    .appName("Homology search")
    .config("spark.executor.cores","8")
    .config("spark.executor.memory", "19G")
    .getOrCreate()
// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._
spark.sparkContext.setLogLevel("WARN")

import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SparkSession
import org.bdgenomics.adam.rdd.ADAMContext._
import comp.bio.aging.playground.extensions._
import org.apache.spark.sql._
import scala.util.Try
import org.apache.spark.storage.StorageLevel
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@58168a28
import spark.implicits._


In [ ]:
def readTSV(path: String) = spark.read.option("sep", "\t").csv(path)

val base = "hdfs://namenode/pipelines"
val human = base + "/HUMAN/GRCh38/27"
val genomeAdam = human + "/genome.adam"
val chrYAdam = human + "/chrY.adam"
  

readTSV: (path: String)org.apache.spark.sql.DataFrame
base: String = hdfs://namenode/pipelines
human: String = hdfs://namenode/pipelines/HUMAN/GRCh38/27
genomeAdam: String = hdfs://namenode/pipelines/HUMAN/GRCh38/27/genome.adam
chrYAdam: String = hdfs://namenode/pipelines/HUMAN/GRCh38/27/chrY.adam


In [ ]:
val genome = sparkContext.loadParquetContigFragments(genomeAdam)
val contigNames = genome.sequences.records.map(_.name)
contigNames


genome: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 555 reference sequences
contigNames: scala.collection.immutable.Vector[String] = Vector(GL383554.1, KN538367.1, KI270579.1, KV575246.1, KI270733.1, GL383532.1, KI270869.1, KI270765.1, KI270890.1, KI270849.1, KQ090017.1, KQ090024.1, KI270820.1, KI270915.1, KN538370.1, GL383521.1, KI270742.1, KB021644.2, KI270829.1, KV575256.1, KV575250.1, KI270583.1, KI270870.1, chr1, GL000251.2, KI270868.1, GL000219.1, KI270317.1, KI270744.1, KI270921.1, KI270747.1, KI270787.1, KI270783.1, KV575259.1, KQ090022.1, KI270936.1, chr4, KV880765.1, KV575252.1, GL383575.2, KN538368.1, GL383518.1, KI270904.1, KI270335.1, chrX, chr20, KI270735.1, KQ090023.1, KI270374.1, GL383530.1, KI270826.1, chr1...

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
import org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD
def saveHuman(frags: NucleotideContigFragmentRDD,name: String) = {
  val path = if(name.startsWith("/")) human + name else human + "//" + name
  frags.saveAsParquet(path)
  path  
}

import org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD
saveHuman: (frags: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD, name: String)String


In [ ]:
val guidomePamAdam =       human + "/guidomePam.adam"
val guidomeNoPamAdam =     human + "/guidomeNoPam.adam"
val guidomeChrYPamAdam = human + "/guidomeChrYPam.adam"
val guidomeChrYNoPamAdam =   human + "/guidomeChrYNoPam.adam"

guidomePamAdam: String = hdfs://namenode/pipelines/HUMAN/GRCh38/27/guidomePam.adam
guidomeNoPamAdam: String = hdfs://namenode/pipelines/HUMAN/GRCh38/27/guidomeNoPam.adam
guidomeChrYPamAdam: String = hdfs://namenode/pipelines/HUMAN/GRCh38/27/guidomeChrYPam.adam
guidomeChrYNoPamAdam: String = hdfs://namenode/pipelines/HUMAN/GRCh38/27/guidomeChrYNoPam.adam


In [ ]:
import comp.bio.aging.crispr._
val cas9 = new Cas9ADAM()

//val chrY = genome.transform(rdd=>rdd.filter(_.getContigName=="chrY"))
//saveHuman(chrY, "/chrY.adam")
//val guidomePam = cas9.guidome(genome, true)
//val guidomeNoPam = cas9.guidome(genome, false)

val guidomePam = sparkContext.loadParquetContigFragments(guidomePamAdam)
val guidomeNoPam = sparkContext.loadParquetContigFragments(guidomeNoPamAdam)

val chrY = sparkContext.loadParquetContigFragments(chrYAdam)

//val chrYNoPam = cas9.guidome(chrY, false)
//val chrYPam = cas9.guidome(chrY, true)

val chrYPam = sparkContext.loadParquetContigFragments(guidomeChrYPamAdam)
val chrYNoPam = sparkContext.loadParquetContigFragments(guidomeChrYNoPamAdam)

import comp.bio.aging.crispr._
cas9: comp.bio.aging.crispr.Cas9ADAM = comp.bio.aging.crispr.Cas9ADAM@27c9c86c
guidomePam: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 555 reference sequences
guidomeNoPam: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 555 reference sequences
chrY: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 555 reference sequences
chrYPam: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 555 reference sequences
chrYNoPam: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with...

In [ ]:
val cho = base + "/CHO"
val genomeCHOfna = cho + "/GCA_900186095.1_CHOK1S_HZDv1_genomic.fna"
val choAdam = cho +"/genome_CHO.adam"
val genomeCHO = sparkContext.loadParquetContigFragments(choAdam)

cho: String = hdfs://namenode/pipelines/CHO
genomeCHOfna: String = hdfs://namenode/pipelines/CHO/GCA_900186095.1_CHOK1S_HZDv1_genomic.fna
choAdam: String = hdfs://namenode/pipelines/CHO/genome_CHO.adam
genomeCHO: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 8264 reference sequences


In [ ]:
//val guidomePamCHO = cas9.guidome(genomeCHO, true)
//val guidomeNoPamCHO = cas9.guidome(genomeCHO, false)
val guidomePamAdamCHO =   cho + "/guidomePamCHO.adam"
val guidomeNoPamAdamCHO =   cho + "/guidomeNoPamCHO.adam"
//guidomePamCHO.saveAsParquet(guidomePamAdamCHO)
//guidomeNoPamCHO.saveAsParquet(guidomeNoPamAdamCHO)
val guidomePamCHO = sparkContext.loadParquetContigFragments(guidomePamAdamCHO)
val guidomeNoPamCHO = sparkContext.loadParquetContigFragments(guidomeNoPamAdamCHO)

guidomePamAdamCHO: String = hdfs://namenode/pipelines/CHO/guidomePamCHO.adam
guidomeNoPamAdamCHO: String = hdfs://namenode/pipelines/CHO/guidomeNoPamCHO.adam
guidomePamCHO: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 8264 reference sequences
guidomeNoPamCHO: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 8264 reference sequences


In [ ]:
val yReps = Rainbow.countRepeats(chrYNoPam, 10).cache
yReps.count

yReps: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[12] at filter at Rainbow.scala:24
res15: Long = 4435


4435

In [ ]:
val noY = guidomeNoPam.transform(rdd=>rdd.filter(c=>c.getContigName!="chrY"))
val noYrepeats5 = Rainbow.countRepeatsByContig(noY, 5).persist(StorageLevel.MEMORY_AND_DISK)
noYrepeats5.count


noY: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = RDDBoundNucleotideContigFragmentRDD with 555 reference sequences
noYrepeats5: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[10] at filter at Rainbow.scala:29
res9: Long = 859593


859593

In [ ]:
val repsOnlyY= yReps.keys.subtract(noYrepeats5.keys.map(_._1)).cache()
repsOnlyY.count()
  

repsOnlyY: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[17] at subtract at <console>:102
res11: Long = 1285


1285

In [ ]:
//repsOnlyY.saveAsTextFile(human + "/y_only_repeats_10.txt")
//yReps.saveAsTextFile(human + "/all_y_repeats_10.txt")
yReps.take(10)

res21: Array[(String, Int)] = Array((TCTCACTCTGTTGCCCAGGC,59), (TCTAGGGTTTTTATGGTTTT,84), (GGAGACCAGCCTGGCCAACA,18), (CTAAAAATACAAAATTAGCC,46), (TGGAATGGAATCGAATGCAA,28), (ATTAGGACGTGGACATCTTT,62), (TCTTTAAGAATGTTGAATAT,62), (TGGATTCGAATGGAAGGCAA,24), (GAAGGCCTTTTCTCGGCAGC,11), (TGTAATCCCAGCACTTTAGG,16))


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
val repeatsCHO = Rainbow.countRepeatsByContig(guidomeNoPamCHO, 2).persist(StorageLevel.MEMORY_AND_DISK)
repeatsCHO.count

repeatsCHO: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[90] at filter at Rainbow.scala:29
res76: Long = 2228145


2228145

In [ ]:
val guidesY = sparkContext.textFile(human + "/y_only_repeats_10.txt")


guidesY: org.apache.spark.rdd.RDD[String] = hdfs://namenode/pipelines/HUMAN/GRCh38/27/y_only_repeats_10.txt MapPartitionsRDD[19] at textFile at <console>:92


In [ ]:
val trueUnique = guidesY.subtract(repeatsCHO.keys.map(_._1.toUpperCase))
trueUnique.count

trueUnique: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[98] at subtract at <console>:103
res80: Long = 1285


1285

In [ ]:
repeatsCHO.keys.map(_._1.toUpperCase).take(10)

res78: Array[String] = Array(TGTGGCTTTGGAGGCTGCCC, CCTGGGGAGCAGAAAAGGTA, AAACTTGTAGAGGAAGATTA, GGAACCTGTGTTAGGTAAAG, ACCTCCTAGTGACTAAGACA, ATTTTGTGTTAGTGATTTGT, TAATTCTTTATTGATTCTTT, GACCCTATCATGTGTAAAGC, GGTGGTTGTGAGCCACTATG, TAGACTTTGATGACACCCTA)


entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
def selectTargets(guidesY: RDD[String],
                  guidome: NucleotideContigFragmentRDD,
                  minCluster: Long = 10,
                  maxDist: Long = 1000) = {
  val strs = guidesY.collect()
  guidome.rdd
    .filter(f=>strs.contains(f.getSequence))
    .map(f=>f.getSequence->f)
    .aggregateByKey(List.empty[(Long, Long)])(
    {
      case (Nil, frag) => (0L, frag.getStart.toLong)::Nil
      case ((n, last)::tail, frag) =>
        if(Math.abs(frag.getStart.toLong - last) > maxDist)
          (0L, frag.getStart.toLong)::(n, last)::tail
        else (n + 1L, frag.getStart.toLong)::tail
    },
    {case (a, b) => a ++ b}
    )
    .mapValues(l=> l.collect{ case (n, _) if n >= minCluster => n })
    .filter(_._2.nonEmpty)
}

selectTargets: (guidesY: org.apache.spark.rdd.RDD[String], guidome: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD, minCluster: Long, maxDist: Long)org.apache.spark.rdd.RDD[(String, List[Long])]


In [ ]:
val targets = selectTargets(trueUnique,chrYNoPam, 10, 800).cache
targets.count

targets: org.apache.spark.rdd.RDD[(String, List[Long])] = MapPartitionsRDD[78] at filter at <console>:113
res68: Long = 4


4

In [ ]:
targets.collect

res70: Array[(String, List[Long])] = Array((AAGACCATGAACCTGCCAGA,List(15, 34, 19)), (CTGCCAGAAGGAAGAAACTC,List(11, 12)), (GAAACTCTGGACACATCTGA,List(10)), (AATGAAGCTGCAGACCTTAC,List(33, 42, 19, 26, 22)))


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
val c  = targets.map(_._1).collect().toSet
//c
yReps.filter(kv=>c.contains(kv._1)).collect
//yReps.take(1)


c: scala.collection.immutable.Set[String] = Set(CAGGGTTTCTGTCTTCTGGC, GAAACTCTGGACACATCTGA, AAGACCATGAACCTGCCAGA, CTGCCAGAAGGAAGAAACTC, AATGAAGCTGCAGACCTTAC, TGTCCAGGGTTTCTGTCTTC, TGTCCAGTGTTTCTTCCTTC)
res56: Array[(String, Int)] = Array((CAGGGTTTCTGTCTTCTGGC,63), (TGTCCAGGGTTTCTGTCTTC,71), (AAGACCATGAACCTGCCAGA,71), (CTGCCAGAAGGAAGAAACTC,45), (GAAACTCTGGACACATCTGA,36), (AATGAAGCTGCAGACCTTAC,182), (TGTCCAGTGTTTCTTCCTTC,50))


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
//saveHuman(guidomeNoPam, "/guidomeNoPam.adam")
//saveHuman(chrYNoPam, "/guidomeChrYNoPam.adam")
//saveHuman(chrYPam, "/guidomeChrYPam.adam")
//saveHuman(guidomeNoPam, "/guidomeNoPam.adam")


res32: String = hdfs://namenode/pipelines/HUMAN/GRCh38/27/guidomeNoPam.adam


hdfs://namenode/pipelines/HUMAN/GRCh38/27/guidomeNoPam.adam

In [ ]:
import better.files._
import java.io.{File => JFile}
val pipelines = better.files.File("/pipelines")
val indexes = pipelines / "indexes"

import better.files._
import java.io.{File=>JFile}
pipelines: better.files.File = /pipelines
indexes: better.files.File = /pipelines/indexes


In [ ]:
val human = indexes / "HUMAN" / "27"
human.list.toList

human: better.files.File = /pipelines/indexes/HUMAN/27
res10: List[better.files.File] = List(/pipelines/indexes/HUMAN/27/convert.sh, /pipelines/indexes/HUMAN/27/gencode.v27.pc_transcripts.fa, /pipelines/indexes/HUMAN/27/gencode.v27.transcripts.fa, /pipelines/indexes/HUMAN/27/gencode.v27.pc_translations.fa, /pipelines/indexes/HUMAN/27/gencode.v27.lncRNA_transcripts.fa, /pipelines/indexes/HUMAN/27/GRCh38.p10.genome.fa, /pipelines/indexes/HUMAN/27/log.out)


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total